In [ ]:
import torch
import numpy as np
import os
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# 저장된 prediction test

In [ ]:
model_name = 'bert-base-uncased'

train_df = pd.read_pickle(os.path.join(f'{model_name}_dataset', 'train.pkl'))
val_df = pd.read_pickle(os.path.join(f'{model_name}_dataset', 'val.pkl'))
# test_df = pd.read_pickle(os.path.join(f'{model_name}_dataset', 'test.pkl'))

train_true_label = train_df['label']
val_true_label = val_df['label']
# test_true_label = test_df['label']

In [ ]:
def label_acc(df, predict, label):
    index = df.loc[df['label']==label].index
    return (predict[index] == label).sum() / len(index)

from collections import Counter

def document_acc(df, predict):
    is_correct = []
    docs_index = df['docs_index'].drop_duplicates()
    for i in docs_index:
        index = df.loc[df['docs_index']==i].index
        docs_label = df.loc[df['docs_index']==i]['label'].iloc[0]
        predicted_label = Counter(predict[index]).most_common(1)[0][0]
        is_correct.append(predicted_label==docs_label)
    
    return np.array(is_correct).sum() / len(is_correct)

In [ ]:
train_f1 = []
train_acc = {
    'label0': [],
    'label1': [],
    'label2': [],
    'whole': [],
    'docs': []
}
val_f1 = []
val_acc = {
    'label0': [],
    'label1': [],
    'label2': [],
    'whole': [],
    'docs': []
}

for i in range(20):
    valid_predict = torch.load(os.path.join(f'{model_name}_val_predictions', f'{i}valid.pkl'), map_location=torch.device('cpu'))
    val_flatten_predictions = np.stack([torch.softmax(torch.Tensor(p), dim=0).cpu().detach().numpy() for batch in valid_predict for p in batch])

    valid_predicts_label = np.argmax(val_flatten_predictions, axis=1)

    val_f1.append(f1_score(val_true_label, valid_predicts_label, average='macro'))
    val_acc['whole'].append(accuracy_score(val_true_label, valid_predicts_label))

    val_acc['docs'].append(document_acc(val_df, valid_predicts_label))

    print(f"{i}번째 점수")
    print(f"{i}train f1: {train_f1[i]}")
    print(f"{i}val f1: {val_f1[i]}")
    print(f"{i}train acc: {train_acc['whole'][i]}")
    print(f"{i}val acc: {val_acc['whole'][i]}")
    print()